Instalando Pacote

In [1]:
!git clone https://github.com/huggingface/transformers;
!cd transformers; pip3 install .

Cloning into 'transformers'...
remote: Enumerating objects: 97246, done.
remote: Counting objects: 100% (378/378), done.
remote: Compressing objects: 100% (286/286), done.
remote: Total 97246 (delta 160), reused 181 (delta 70), pack-reused 96868
Receiving objects: 100% (97246/97246), 90.22 MiB | 20.57 MiB/s, done.
Resolving deltas: 100% (71424/71424), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel meta

In [2]:
! pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 14.2 MB/s 
     |████████████████████████████████| 1.8 MB 50.8 MB/s 
     |████████████████████████████████| 10.1 MB 67.3 MB/s 
     |████████████████████████████████| 1.2 MB 58.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 346 kB 54.0 MB/s 
     |████████████████████████████████| 145 kB 72.4 MB/s 
     |████████████████████████████████| 181 kB 70.8 MB/s 
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 140 kB 31.0 MB/s 
     |████████████████████████████████| 212 kB 55.5 MB/s 
     |████████████████████████████████| 1.1 MB 14.9 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
     |████████████████████████████████| 127 kB 59.5 MB/s 
     |████████████████████████████████| 271 kB 10.6 MB/s 
     |██████████████████████████████

In [3]:
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Puxando Dataset de Discurso de Ódio

In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import pandas as pd
import transformers
from transformers import AutoModel, BertTokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer



# link: https://huggingface.co/datasets/ag_news

In [11]:
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/paper_asoc/SemEval2018-T3-train-taskA_emoji.txt',sep = '\t')
dataset.head()

,Tweet index,Label,Tweet text
0,1,1,Sweet United Nations video. Just in time for C...
1,2,1,@mrdahl87 We are rumored to have talked to Erv...
2,3,1,Hey there! Nice to see you Minnesota/ND Winter...
3,4,0,3 episodes left I'm dying over here
4,5,1,I can't breathe! was chosen as the most notabl...


In [12]:
dataset_teste = pd.read_csv('/content/drive/My Drive/Colab Notebooks/paper_asoc/SemEval2018-T3_gold_test_taskA_emoji.txt',sep = '\t')
dataset_teste.head()

,Tweet index,Label,Tweet text
0,1,0,@Callisto1947 Can U Help?||More conservatives ...
1,2,1,"Just walked in to #Starbucks and asked for a ""..."
2,3,0,#NOT GONNA WIN http://t.co/Mc9ebqjAqj
3,4,0,@mickymantell He is exactly that sort of perso...
4,5,1,So much #sarcasm at work mate 10/10 #boring 10...


In [13]:
X_train = dataset['Tweet text']
X_test = dataset_teste['Tweet text']
y_train = dataset['Label']
y_test = dataset_teste['Label']

In [14]:
# Quatro classes

# 0:"World"
# 1:"Sports"
# 2:"Business"
# 3:"Sci/Tech"


In [15]:
def puxa_dados_normal(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')
  original = csv[['texto', 'label']].copy()
  original = original.drop_duplicates()
  
  # csv = csv[csv['similiaridade_parafrase'] >= 0.95].copy()
  

  parafrase = csv[['parafraseador', 'label']].copy()
  parafrase = parafrase.drop_duplicates()
  parafrase = parafrase[~(parafrase['parafraseador'].isnull())].copy()
  parafrase.rename(columns={"parafraseador": "texto"}, inplace = True)

  fim = original.append(parafrase)

  fim = fim.sample(frac=1)

  X = fim['texto'].values
  y = fim['label'].values
  

  return X, y 


In [16]:
def desempenho_LSTM(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  mask_zero= True,
                                  output_dim=embedding_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  
  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  
  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [17]:
def desempenho_CNN(X_train, X_test, y_train, y_test, classes, num_epochs ):


  # vocab_size = 10000
  embedding_dim = 300
  max_length = 128
  trunc_type = 'post'
  padding_type = 'post'
  oov_tok = '<OOV>'

  tokenizer = Tokenizer( oov_token=oov_tok)
  tokenizer.fit_on_texts(X_train)
  word_index = tokenizer.word_index
  vocab_size = len(word_index)+1

  train_sequences = tokenizer.texts_to_sequences(X_train)
  # val_sequences = tokenizer.texts_to_sequences(X_val)

  train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
  # val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  # model = tf.keras.Sequential([
  #   # Camada Embedding (reduzir dimensionalidade)
  #   tf.keras.layers.Embedding(vocab_size, embedding_dim),
  #   # Camada de LSTM bidirecional
  #   tf.keras.layers.LSTM(embedding_dim),
  #   # Camada densa com Relu
  #   tf.keras.layers.Dense(embedding_dim, activation='relu'),
  #   #Camada de saída para as 4 classes
  #   tf.keras.layers.Dense(classes, activation='softmax')
  #   ])
  model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=vocab_size, 
                                  mask_zero= True,
                                  output_dim=embedding_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Conv1D(filters=100, kernel_size = 3, activation = 'relu', 
                               # set 'axis' value to the first and second axis of conv1D weights (rows, cols)
                               kernel_constraint= MaxNorm( max_value=3, axis=[0,1])),
        
        tf.keras.layers.MaxPool1D(2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu', 
                              # set axis to 0 to constrain each weight vector of length (input_dim,) in dense layer
                              kernel_constraint = MaxNorm( max_value=3, axis=0)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=classes, activation='softmax')
    ])
  # model = tf.keras.Sequential()
  # model.add(tf.keras.layers.Embedding(vocab_size, embedding_dim))
  # model.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
  # model.add(tf.keras.layers.MaxPooling1D(pool_size=2))
  # model.add(tf.keras.layers.LSTM(100))
  # model.add(tf.keras.layers.Dense(classes, activation='softmax'))
  
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  
  history = model.fit(train_padded, y_train, epochs=num_epochs, verbose=2)

  test_sequences = tokenizer.texts_to_sequences(X_test)
  test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  
  preds_log = model.predict(test_padded)
  preds = tf.argmax(preds_log, 1)
  acc = accuracy_score(y_test, preds)
  f1 = f1_score(y_test, preds, average='macro')

  return acc, f1







In [18]:
# Evaluate the model
from sklearn.metrics import f1_score, accuracy_score

def f1_multiclass(labels, preds):
      return f1_score(labels, preds, average='macro')


In [19]:
def desempenho_transformers(X_train, X_test, y_train, y_test,classes, model,type_model, epochs, batch, max_len):
  from simpletransformers.classification import ClassificationModel
  import pandas as pd
  import logging
  import sklearn

  train_df = pd.DataFrame(X_train)
  train_df['target'] = y_train

  eval_df = pd.DataFrame(X_test)
  eval_df['target'] = y_test



  # Create a ClassificationModel
  model = ClassificationModel(type_model, model, num_labels=classes, args={'num_train_epochs':epochs,
                                                                           'train_batch_size':batch,
                                                                           'max_seq_length':max_len,
                                                                          #  "fp16":False,
                                                                           'overwrite_output_dir': True}) 

  # Train the model
  model.train_model(train_df)
  result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score,f1=f1_multiclass)

  return result['acc'], result['f1']





In [20]:
def desempenho_SVM(X_train, X_test, y_train, y_text):
  text_clf_svm = Pipeline([('vect', CountVectorizer()),
                        ('tfidf', TfidfTransformer()),
                        ('clf-svm', SVC(gamma='auto')),])
  _ = text_clf_svm.fit(X_train, y_train)
  predicted_svm = text_clf_svm.predict(X_test)
  acc = accuracy_score(y_test, predicted_svm)
  f1 = f1_score(y_test, predicted_svm, average='macro')

  return acc, f1


In [21]:

# X_test = dataset['test']['text']
# y_test = dataset['test']['label-coarse']
y_test = np.array(y_test)


In [22]:
y_test.shape

(784,)

In [23]:
def puxa_original(caminho):
  csv = pd.read_csv(caminho, sep = ';', decimal = ',')

  X = csv['texto'].values
  y = csv['label'].values
  

  return X, y 

In [40]:
perc = 50
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  # if j == 0:
  #   X_train,  y_train = puxa_original(arqs[j])
  # else:
  X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 2, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_50/base_ironic_50_ssmba1.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

  0%|          | 0/11448 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/358 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/784 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/98 [00:00<?, ?it/s]

BERT
ACC:0.6696428571428571
F1:0.6680089792701809


In [41]:
data = {'Aug': [ 'ssmba1', 'ssmba2', 'ssmba3', 'ssmba4','ssmba5','ssmba6', 'ssmba7', 'ssmba8', 'ssmba9', 'ssmba10'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)

In [42]:
base_resultados

,Aug,BERT_ACC,BERT_F1
0,ssmba1,0.701531,0.697895
1,ssmba2,0.664541,0.661607
2,ssmba3,0.650510,0.644963
3,ssmba4,0.664541,0.663331
4,ssmba5,0.656888,0.652999
5,ssmba6,0.655612,0.645565
6,ssmba7,0.668367,0.664328
7,ssmba8,0.697704,0.690076
8,ssmba9,0.670918,0.660709
9,ssmba10,0.669643,0.668009


In [43]:
# repete = 1
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_resultados_ssmba.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

In [44]:
perc = 75
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  # if j == 0:
  #   X_train,  y_train = puxa_original(arqs[j])
  # else:
  X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 2, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])
data = {'Aug': [ 'ssmba1', 'ssmba2', 'ssmba3', 'ssmba4','ssmba5','ssmba6', 'ssmba7', 'ssmba8', 'ssmba9', 'ssmba10'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)

# repete = 1
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_resultados_ssmba.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_75/base_ironic_75_ssmba1.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

  0%|          | 0/17166 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/537 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/784 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/98 [00:00<?, ?it/s]

BERT
ACC:0.7066326530612245
F1:0.7054345624317984


In [45]:
perc = 100
i=0
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm
arqs = [
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
        '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
       '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_ssmba'+str(i+1)+'.csv',
      # # '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_original.csv',
       ]

svm_acc = []
svm_f1 = []
lstm_acc = []
lstm_f1 = []
bert_acc = []
bert_f1 = []
xlnet_acc = []
xlnet_f1 = []
import gc
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
torch.cuda.empty_cache()
gc.collect()
for j in range(len(arqs)):
  print(arqs[j])
  # if j == 0:
  #   X_train,  y_train = puxa_original(arqs[j])
  # else:
  X_train,  y_train = puxa_dados_normal(arqs[j])
  # svm = desempenho_CNN(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('CNN')
  # print('ACC:' + str(svm[0]))
  # print('F1:' + str(svm[1]))
  # lstm = desempenho_LSTM(X_train,  X_test, y_train,  y_test, 6, 10)
  # print('LSTM')
  # print('ACC:' + str(lstm[0]))
  # print('F1:' + str(lstm[1]))
  torch.cuda.empty_cache()
  gc.collect()
  bert = desempenho_transformers(X_train, X_test, y_train, y_test, 2, 'distilbert-base-uncased','distilbert', 1, 32, 64)
  
  print('BERT')
  print('ACC:' + str(bert[0]))
  print('F1:' + str(bert[1]))

  # torch.cuda.empty_cache()
  # gc.collect()

  # xlnet = desempenho_transformers(X_train, X_test, y_train, y_test, 6, 'xlnet-base-cased','xlnet',  4, 32, 128)
  # print('XlNet')
  # print('ACC:' + str(xlnet[0]))
  # print('F1:' + str(xlnet[1]))

  # svm_acc.append(svm[0])
  # svm_f1.append(svm[1])

  # lstm_acc.append(lstm[0])
  # lstm_f1.append(lstm[1])

  bert_acc.append(bert[0])
  bert_f1.append(bert[1])

  # xlnet_acc.append(xlnet[0])
  # xlnet_f1.append(xlnet[1])
data = {'Aug': [ 'ssmba1', 'ssmba2', 'ssmba3', 'ssmba4','ssmba5','ssmba6', 'ssmba7', 'ssmba8', 'ssmba9', 'ssmba10'],
       'BERT_ACC': bert_acc, 'BERT_F1':bert_f1,} # ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }

# data = {'Aug': ['bert', ],'sem', 'key', 'eda', 'emb', 'bert',, 'pegasus'
#         'SVM_ACC': svm_acc, 'SVM_F1':svm_f1,'LSTM_ACC': lstm_acc, 'LSTM_F1':lstm_f1 ,'BERT_ACC': bert_acc, 'BERT_F1':bert_f1, 'XLNET_ACC': xlnet_acc, 'XLNET_F1':xlnet_f1 }


base_resultados = pd.DataFrame.from_dict(data)

# repete = 1
arquivo = '/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_' +str(perc) + '/base_ironic_'+str(perc)+'_resultados_ssmba.csv'
base_resultados.to_csv(arquivo, encoding='utf-8', index=False, sep=';', decimal = ',')

/content/drive/My Drive/Colab Notebooks/paper_asoc/perc_100/base_ironic_100_ssmba1.csv


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

  0%|          | 0/22878 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Epoch 0 of 1:   0%|          | 0/715 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/784 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/98 [00:00<?, ?it/s]

BERT
ACC:0.7142857142857143
F1:0.7089655629578071
